In [ ]:
import requests
import json
import os
from tqdm import tqdm
import logging
import time

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

keywords = ['Premium+', 'Premium', 'Bauteam', 'Spieler', 'Entwickler', 'VIP', 'Content', 'Supporter', 'Owner', 'Moderator', 'Translator']

player_data = {}
player_categories = {keyword: [] for keyword in keywords}

def fetch_player_data(player_name, log=False):
    url = f"https://api.cytooxien.de/user/{player_name}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if log: 
            logging.info(f"Fetching API data for player: {player_name}")
        
        # Initialize player data structure
        player_data[player_name] = {
            "name": player_name,
            "rank": "None",
            "minecraft_party": {}
        }
        
        # Extract rank information
        if 'playerInfo' in data and 'rank' in data['playerInfo']:
            rank_name = data['playerInfo']['rank']['name']
            if rank_name in keywords:
                player_data[player_name]["rank"] = rank_name
                player_categories[rank_name].append(player_name)
                if log: 
                    logging.info(f"Player rank: {rank_name}")
            else:
                player_data[player_name]["rank"] = rank_name
        
        # Extract Minecraft Party data (mp stats)
        if 'stats' in data and 'mp' in data['stats'] and 'global' in data['stats']['mp']:

            mp_stats = data['stats']['mp']['global']
            
            # Calculate win rates
            wins = mp_stats.get('wins', 0)
            games = mp_stats.get('games', 0)
            minigame_wins = mp_stats.get('minigame_wins', 0)
            minigames = mp_stats.get('minigames', 0)
            
            winrate = round((wins / games * 100), 2) if games > 0 else 0
            minigame_winrate = round((minigame_wins / minigames * 100), 2) if minigames > 0 else 0
            
            stat_mapping = {
                'Gewonnene Spiele': wins,
                'Gewonnene Minispiele': minigame_wins,
                'Gespielte Spiele': games,
                'Gespielte Minispiele': minigames,
                'Spielzeit': format_time(mp_stats.get('time', 0)),
                'Punkte': mp_stats.get('points', 0),
                'Rang': mp_stats.get('rank_points', 0),
                'Winrate %': winrate,
                'Minigame Winrate %': minigame_winrate
            }
            
            player_data[player_name]["minecraft_party"] = stat_mapping
            
            if log: 
                logging.info("Found Minecraft Party data")
                for stat_name, stat_value in stat_mapping.items():
                    logging.info(f"Scraped: {stat_name} = {stat_value}")
        else:
            if log:
                logging.warning(f"No Minecraft Party data found for {player_name}")
        
        return player_data[player_name]["rank"]
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching data for {player_name}: {e}")
        player_data[player_name] = {
            "name": player_name,
            "rank": "Error",
            "minecraft_party": {}
        }
        return "Error"
    except json.JSONDecodeError as e:
        logging.error(f"Error parsing JSON for {player_name}: {e}")
        player_data[player_name] = {
            "name": player_name,
            "rank": "Error",
            "minecraft_party": {}
        }
        return "Error"

def format_time(milliseconds):
    # Convert milliseconds to a readable time format
    if milliseconds == 0:
        return "0"
    
    seconds = milliseconds // 1000
    minutes = seconds // 60
    hours = minutes // 60
    days = hours // 24
    
    if days > 0:
        return f"{days}d {hours % 24}h {minutes % 60}m"
    elif hours > 0:
        return f"{hours}h {minutes % 60}m"
    elif minutes > 0:
        return f"{minutes}m {seconds % 60}s"
    else:
        return f"{seconds}s"

/Users/eliasmb/MCP-Records/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import helpers as h
playerList = h.assemblePlayerList()

# Fetch data for all players with progress bar
progress_bar = tqdm(playerList, desc="Fetching Progress", unit="player")
for player_name in progress_bar:
    last_rank = fetch_player_data(player_name, log=False)
    progress_bar.set_postfix(last_rank=last_rank, last_player=player_name)
    time.sleep(0.1)  # Small delay to be respectful to the API

# Export data as JSON
with open('player_data.json', 'w', encoding='utf-8') as f:
    json.dump(player_data, f, ensure_ascii=False, indent=4)

print("Data exported to player_data.json")

Processing 242 players...
Found 242 unique UUIDs
Progress: 1/242 - BastiGHG
Progress: 2/242 - DrSpeed06
Progress: 3/242 - GlowyDusk
Progress: 4/242 - ReichesBrot
Progress: 5/242 - SB387
Progress: 6/242 - NotLennart
Progress: 7/242 - AnzeigeGehtRaus
Progress: 8/242 - JOW23
Progress: 9/242 - Shiewk
Progress: 10/242 - Lingex
Progress: 11/242 - Gryzes
Progress: 12/242 - ByNetherdude
Progress: 13/242 - Schmaarek
Progress: 14/242 - xiooh
Progress: 15/242 - LucaaOn
Progress: 16/242 - WuschigesEtwas
Progress: 17/242 - Luxemburq
Progress: 18/242 - Hyxeed
Progress: 19/242 - Grapfen
Progress: 20/242 - Cytoox
Progress: 21/242 - 2B9
Progress: 22/242 - Gestimus
Progress: 23/242 - OutcroYoutube
Progress: 24/242 - Morittz
Progress: 25/242 - FluffyDragon2007
Progress: 26/242 - Nor_Malo
Progress: 27/242 - Wolkenfarmer
Progress: 28/242 - Navex
Progress: 29/242 - Sey__
Progress: 30/242 - Kyuudo
Progress: 31/242 - zBro
Progress: 32/242 - SmartyxD
Progress: 33/242 - Gfrasti
Progress: 34/242 - h4nnes
Progres

Fetching Progress: 100%|██████████| 242/242 [06:13<00:00,  1.54s/player, last_player=zLachs, last_rank=Premium]          

Data exported to player_data.json
